# Business Location Data Analysis

### Imports

In [ ]:
# Packages
import pandas as pd
import configparser
import importlib
from sklearn.preprocessing import MultiLabelBinarizer

# Module
import funcs
importlib.reload(funcs)

### Constants

In [ ]:
# Read Config file
config = configparser.ConfigParser()
config.read('config.ini')

In [ ]:
# Parameter for API call - adjust as needed
# Distance in meters from location specified to search
RADIUS = 10000
# OR use rankby and type
RANKBY = None
# Filter by type of place
TYPE = 'cafe'

# Get private params from config
API_KEY = config['DETAILS']['API_KEY']
LOCATION = config['DETAILS']['LOCATION']

# Not needed columns
DROP_COLS = ['icon', 'icon_background_color', 'icon_mask_base_uri', 'photos', 'reference', 'scope', 'plus_code']

### Make API Call

In [ ]:
results = funcs.make_request(API_KEY, LOCATION, radius = RADIUS, typ = TYPE, rankby = RANKBY)

In [ ]:
# Take a look at the data
results

## Clean Data

In [ ]:
# Check how many results we have
len(results)

In [ ]:
# Return duplicated places
results[results['name'].duplicated()]

In [ ]:
# Reset Index
results = results.reset_index(drop=True)

In [ ]:
# Drop columns we don't need
results = results.drop(DROP_COLS, axis=1)

In [ ]:
# Seperate latitude and logitude values in geometery column into two columns
results['lat'] = results['geometry'].apply(lambda x: x['location']['lat'])
results['long'] = results['geometry'].apply(lambda x: x['location']['lng'])

# Drop geometry column
results = results.drop('geometry', axis=1)

In [ ]:
# Convert types column to one hot encoded columns
mlb = MultiLabelBinarizer(sparse_output=True)

df = results.join(
            pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(results.pop('types')),
                index=results.index,
                columns=mlb.classes_))

In [ ]:
df

## Analysis
- Count by type
- Distance from location
- Visualise on map by type
- Identify categories of interest
- Ratings & number of reviews of those categories
- Visualise by type where category of interest
- Use Place Details Atmosphere to get more details on locations of interest
- Re-run for different radiuses and locations that customers may come from - stretham, haddenham, witchford, cottenham
- Get population & demographic data - use schools, old peoples homes etc and search for public datasets

In [ ]:
# Get columns of types
types_columns = df.loc[:,'long':]
types_columns = types_columns.drop('long', axis=1)
types_columns = types_columns.columns

In [ ]:
# Value counts of types
df[types_columns].sum()

## Export Data
Export the dataframe to a csv for further analysis. Title it by the parameters used for the search

In [ ]:
# Name of csv
csv_name = f'results_{RADIUS}_{TYPE}.csv'
df.to_csv(csv_name, float_format='%.2f')